In [1]:
import pandas as pd
import nltk
import numpy as np

In [3]:
labeled_words = pd.read_csv('nrc/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', delimiter="\t", header=None)
labeled_words.head()

,0,1,2
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


In [9]:
labeled_words.columns = ['word', 'emotion', 'label']

In [10]:
labeled_words.groupby('word')['label'].sum().to_frame()

,label
word,
aback,0
abacus,1
abandon,3
abandoned,4
abandonment,5
...,...
zone,0
zoo,0
zoological,0


In [119]:
import glob
BOOKS_PATH = 'gutenberg_data/'
all_books = glob.glob(BOOKS_PATH + '*')
with open(all_books[0], 'r') as book:
#     book_sample = pd.DataFrame(book.readlines(), columns=['text'])
    book_sample = pd.read_csv(book, delimiter='\n \n', header=None, names=['text'])
book_sample

<ipython-input-119-dc46ebe2e763>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  book_sample = pd.read_csv(book, delimiter='\n \n', header=None, names=['text'])


,text
0,The Project Gutenberg EBook of The Confessions...
1,This eBook is for the use of anyone anywhere a...
2,almost no restrictions whatsoever. You may co...
3,re-use it under the terms of the Project Guten...
4,with this eBook or online at www.gutenberg.org
...,...
2536,https://www.gutenberg.org
2537,This Web site includes information about Proje...
2538,including how to make donations to the Project...
2539,"Archive Foundation, how to help produce our ne..."


In [142]:
paragraphs = []
par = []
with open(all_books[0], 'r') as paragraphs_file:
    for sentence in paragraphs_file: #one line until the end of file
        if sentence != "\n \n" and sentence != "\n" and sentence != "\n\n" and sentence != '':
            par.append(sentence)
        else:
            paragraphs.append(" ".join(par))
            par = []
        

In [156]:
def get_paragraphs(filename):
    paragraphs = []
    par = []
    with open(filename, 'r') as paragraphs_file:
        for sentence in paragraphs_file: #one line until the end of file
            if sentence != "\n \n" and sentence != "\n" and sentence != "\n\n" and sentence != '':
                par.append(sentence)
            else:
                paragraphs.append(" ".join(par))
                par = []
    return paragraphs

In [154]:
pd.DataFrame(paragraphs, columns=['text'])

'36970'

Finding out when the book actually starts

In [67]:
START_STRING = 'START OF THIS PROJECT GUTENBERG EBOOK'
END_STRING = 'END OF THIS PROJECT GUTENBERG EBOOK'

In [148]:
start_idx = book_sample[book_sample['text'].str.contains(START_STRING)].index[0] + 1
book = book_sample.iloc[start_idx:].reset_index(drop=True)
end_idx = book[book['text'].str.contains(END_STRING)].index[0]
book = book[:end_idx].reset_index(drop=True)
book = book.drop_duplicates(ignore_index=True)

,text
50,In August and September we would capture immen...
51,"In winter, snipe were always numerous on the m..."
52,During the rigour of winter great flocks of mi...
53,"""The water was surrounded with huge nets, fast..."
54,In addition to our nets and snares we had a pr...
55,"A curious little bird, the grebe or dabchick, ..."
56,Chapter 4.\n
57,PARTRIDGE POACHING.\n
58,The bloom on the brambles; the ripening of the...
59,A long life of poaching has not cooled our ard...


Removing newlines (getting paragraphs)

In [150]:
book.drop_duplicates().reset_index(drop=True).iloc[50].to_markdown()

'|      | 50                                                                        |\n|:-----|:--------------------------------------------------------------------------|\n| text | In August and September we would capture immense numbers of               |\n|      |  "flappers"--plump wild ducks--but, as yet, unable to fly. These were     |\n|      |  either caught in the pools, or chased into nets which we set to          |\n|      |  intercept them. As I now took more than my share of the work, and made   |\n|      |  all the gins, springes, and noozes which we used, a rough kind of        |\n|      |  partnership sprung up between us. The young ducks brought us good        |\n|      |  prices, and there was another source of income which paid well, but was  |\n|      |  not of long duration. There is a short period in each year when even the |\n|      |  matured wild ducks are quite unable to fly. The male of the common wild  |\n|      |  duck is called the mallard, and soon after 

Get book id:

# Pipeline

In [252]:
text = []
book_ids = []
paragraph_ids = []
error = []
for book_file in all_books:
    paragraphs = get_paragraphs(book_file)
    book_df = pd.DataFrame(paragraphs, columns=['text'])
    book_id = int(book_file.split('/')[-1].replace('.txt', ''))
    try:
        start_idx = book_df[book_df['text'].str.contains(START_STRING)].index[0] + 1
        book = book_df.iloc[start_idx:].reset_index(drop=True)
        end_idx = book[book['text'].str.contains(END_STRING)].index[0]
        book = book[:end_idx].reset_index(drop=True)
        book = book.drop_duplicates(ignore_index=True)
        for i in range(len(book)):
            book_ids.append(int(book_id))
            paragraph_ids.append(f'{book_id}_{i}')
        text.append(book.values.ravel().tolist())
    except IndexError:
        error.append(book_id)


In [215]:
import numpy as np

In [254]:
dict_ = {
    'book_id': np.array(book_ids),
    'paragraph_id': np.array(paragraph_ids),
    'text': [item for sublist in text for item in sublist]
}

In [255]:
df = pd.DataFrame(dict_)

In [256]:
df.head()

,book_id,paragraph_id,text
0,36970,36970_0,
1,36970,36970_1,"Produced by David Edwards, Linda Hamilton and ..."
2,36970,36970_2,"""Poaching is one of the fine arts--how 'fine' ..."
3,36970,36970_3,[Illustration: THE SQUIRE'S KEEPER.]\n
4,36970,36970_4,The\n Confessions\n of a\n Poacher\n


In [257]:
df.shape

(203334, 3)

In [258]:
df.to_csv('gutenberg_data.csv')